### PromptTemplate 
* [PromptTemplate](https://python.langchain.com/api_reference/core/prompts/langchain_core.prompts.prompt.PromptTemplate.html#langchain_core.prompts.prompt.PromptTemplate)
* [ChatPromptTemplate](https://python.langchain.com/api_reference/core/prompts/langchain_core.prompts.chat.ChatPromptTemplate.html#langchain_core.prompts.chat.ChatPromptTemplate)
* [ChatMessagePromptTemplate](https://python.langchain.com/api_reference/core/prompts/langchain_core.prompts.chat.ChatMessagePromptTemplate.html#langchain_core.prompts.chat.ChatMessagePromptTemplate)
* [FewShotPromptTemplate](https://python.langchain.com/api_reference/core/prompts/langchain_core.prompts.few_shot.FewShotPromptTemplate.html#langchain_core.prompts.few_shot.FewShotPromptTemplate)
* PartialPrompt

In [ ]:
# poetry add python-dotenv langchain langchain-openai

In [1]:
from dotenv import load_dotenv
import os
# .env 파일을 불러와서 환경 변수로 설정
load_dotenv(dotenv_path='../.env')

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
print(OPENAI_API_KEY[:2])

gs


##### 1) PromptTemplate 의 from_template() 함수 사용
* 주로 LLM(텍스트 완성형 모델, ex. Ollama, GPT-3.5)과 함께 사용
* 하나의 문자열 프롬프트를 생성

In [3]:
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from pprint import pprint

template_text = "{model_name} 모델의 학습 원리를 {count} 문장으로 한국어로 답변해 주세요."

# PromptTemplate 인스턴스를 생성
prompt_template = PromptTemplate.from_template(template_text)

# llm = ChatOpenAI(model="gpt-3.5-turbo-0125")
llm = ChatOpenAI(
    api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="openai/gpt-oss-120b",  # Spring AI와 동일한 모델
    temperature=0.7
)

chain = prompt_template | llm | StrOutputParser()
response = chain.invoke({"model_name":"ChatGPT", "count":3})
pprint(response)

('ChatGPT는 대규모 텍스트 데이터를 이용해 **자기지도학습**(self‑supervised learning) 방식으로 사전 학습을 '
 '진행합니다. 이 과정에서 모델은 앞뒤 문맥을 예측하도록 훈련되어, 언어의 통계적 패턴과 의미 관계를 스스로 학습합니다. 이후 특정 작업에 '
 '맞게 **미세조정**(fine‑tuning)이나 **프롬프트 엔지니어링**을 통해 원하는 출력 형태를 구현합니다.')


##### 2) PromptTemplate 결합하기
* 동일한 Prompt 패턴을 사용하지만 여러 개의 질문을 작성해서 LLM을 실행할 수도 있습니다.

In [4]:
template_text = "{model_name} 모델의 학습 원리를 {count} 문장으로 한국어로 답변해 주세요."

# PromptTemplate 인스턴스를 생성
prompt_template = PromptTemplate.from_template(template_text)

# 템플릿에 값을 채워서 프롬프트를 완성
filled_prompt = prompt_template.format(model_name="ChatGPT", count=3)

# 문자열 템플릿 결합 (PromptTemplate + PromptTemplate + 문자열)
combined_prompt = (
              prompt_template
              + PromptTemplate.from_template("\n\n 그리고 {model_name} 모델의 장점을 요약 정리해 주세요")
              + "\n\n {model_name} 모델과 비슷한 AI 모델은 어떤 것이 있나요? 모델명은 {language}로 답변해 주세요."
)
combined_prompt.format(model_name="ChatGPT", count=3, language="영어")
print(combined_prompt)

#llm = ChatOpenAI(model="gpt-3.5-turbo-0125")
chain = combined_prompt | llm | StrOutputParser()
response = chain.invoke({"model_name":"ChatGPT", "count":3, "language":"영어"})

pprint(response)

input_variables=['count', 'language', 'model_name'] input_types={} partial_variables={} template='{model_name} 모델의 학습 원리를 {count} 문장으로 한국어로 답변해 주세요.\n\n 그리고 {model_name} 모델의 장점을 요약 정리해 주세요\n\n {model_name} 모델과 비슷한 AI 모델은 어떤 것이 있나요? 모델명은 {language}로 답변해 주세요.'
('**ChatGPT 모델의 학습 원리 (3문장)**  \n'
 '1. 대규모 텍스트 코퍼스를 사용해 사전 학습(pre‑training) 단계에서 문맥을 예측하는 언어 모델로 먼저 훈련됩니다.  \n'
 '2. 이후 인간이 만든 질문‑답변 쌍이나 지시문을 이용해 강화 학습(RLHF, Reinforcement Learning from '
 'Human Feedback)으로 모델을 미세 조정합니다.  \n'
 '3. 이 두 단계가 결합돼 모델은 다양한 주제와 형식에 대해 자연스럽고 일관된 응답을 생성할 수 있게 됩니다.  \n'
 '\n'
 '**ChatGPT의 주요 장점**  \n'
 '- 방대한 사전 지식과 문맥 이해 능력으로 다양한 분야의 질문에 신속히 답변한다.  \n'
 '- 인간 피드백을 반영한 강화 학습 덕분에 대화 흐름이 자연스럽고 사용자의 의도에 맞는 출력을 제공한다.  \n'
 '- 멀티턴 대화, 코드 생성, 요약, 번역 등 여러 작업을 하나의 모델로 수행할 수 있어 활용도가 높다.  \n'
 '\n'
 '**ChatGPT와 비슷한 AI 모델 (영어명)**  \n'
 '- GPT‑4 (OpenAI)  \n'
 '- LLaMA\u202f2 (Meta)  \n'
 '- Claude\u202f3 (Anthropic)  \n'
 '- Gemini\u202f1.5 (Google DeepMind)  \n'
 '- Mistral‑7B (Mistral AI)  \n'
 '- Falcon‑180B (Tech

#### PromptTemplate 의 파라미터를 배열 형태로 하여 여러개 사용하는 경우

In [5]:
template_text = "{model_name} 모델의 학습 원리를 {count} 문장으로 한국어로 답변해 주세요."

# PromptTemplate 인스턴스를 생성
prompt_template = PromptTemplate.from_template(template_text)

questions = [
    {"model_name": "GPT-4", "count": 3},
    {"model_name": "Gemini", "count": 4},
    {"model_name": "claude", "count": 4},
]

# 여러 개의 프롬프트를 미리 생성
formatted_prompts = [prompt_template.format(**q) for q in questions]
print(formatted_prompts)  # 미리 생성된 질문 목록 확인

#llm = ChatOpenAI(model="gpt-3.5-turbo-0125")

for prompt in formatted_prompts:
    print(type(prompt), prompt)
    response = llm.invoke(prompt)
    pprint(response.content)

['GPT-4 모델의 학습 원리를 3 문장으로 한국어로 답변해 주세요.', 'Gemini 모델의 학습 원리를 4 문장으로 한국어로 답변해 주세요.', 'claude 모델의 학습 원리를 4 문장으로 한국어로 답변해 주세요.']
<class 'str'> GPT-4 모델의 학습 원리를 3 문장으로 한국어로 답변해 주세요.
('GPT‑4는 대규모 텍스트 데이터를 사용해 다음에 올 단어를 예측하도록 학습하는 자기지도 학습 방식을 사용합니다. 이 과정에서 트랜스포머 '
 '아키텍처의 다중‑헤드 어텐션을 통해 문맥을 장거리에 걸쳐 효율적으로 파악하고, 파라미터를 최적화해 언어 패턴을 일반화합니다. 최종 모델은 '
 '사전 학습된 가중치를 기반으로 다양한 다운스트림 작업에 맞게 미세조정하거나 프롬프트 엔지니어링으로 활용됩니다.')
<class 'str'> Gemini 모델의 학습 원리를 4 문장으로 한국어로 답변해 주세요.
('Gemini 모델은 대규모 텍스트와 멀티모달 데이터를 수집해, 트랜스포머 기반의 신경망에 입력합니다.  \n'
 '수집된 데이터를 이용해 **자기 지도 학습**(self‑supervised learning)으로 사전 학습(pre‑training)을 '
 '수행해, 문맥을 예측하거나 이미지‑텍스트 연관성을 학습합니다.  \n'
 '그 후, 특정 작업(예: 질문‑응답, 번역, 이미지 캡션 생성 등)에 맞춰 **지도 학습**(supervised '
 'fine‑tuning)이나 **강화 학습**(RLHF)으로 미세 조정하여 성능을 최적화합니다.  \n'
 '전체 과정은 대규모 병렬 연산을 지원하는 GPU/TPU 클러스터에서 수주에서 수개월에 걸쳐 진행됩니다.')
<class 'str'> claude 모델의 학습 원리를 4 문장으로 한국어로 답변해 주세요.
('Claude 모델은 대규모 텍스트 데이터를 활용해 **사전 학습(Pre‑training)** 단계에서 언어의 통계적 패턴과 문맥 정보를 '
 '학습합니다.  \n'
 '그 후 **지도 학습(Superv

##### 2) ChatPromptTemplate
* Tuple 형태의 system, user, assistant 메시지 지원
* 여러 개의 메시지를 조합하여 LLM에게 전달 가능
* 간결성과 가독성이 높고 단순한 구조

In [6]:
# 2-튜플 형태의 메시지 목록으로 프롬프트 생성 (type, content)

from langchain_core.prompts import ChatPromptTemplate

chat_prompt = ChatPromptTemplate.from_messages([
    # role, message
    ("system", "This system is an expert in answering questions about {topic}. Please provide clear and detailed explanations."),
    ("human", "{model_name} 모델의 학습 원리를 설명해 주세요."),
])

messages = chat_prompt.format_messages(topic="AI", model_name="ChatGPT")
print(messages)

# 생성한 메시지를 바로 주입하여 호출하기
#llm = ChatOpenAI(model="gpt-3.5-turbo-0125")
response = llm.invoke(messages)

print(type(response))
print(response.content)

[SystemMessage(content='This system is an expert in answering questions about AI. Please provide clear and detailed explanations.', additional_kwargs={}, response_metadata={}), HumanMessage(content='ChatGPT 모델의 학습 원리를 설명해 주세요.', additional_kwargs={}, response_metadata={})]
<class 'langchain_core.messages.ai.AIMessage'>
## ChatGPT 모델이 어떻게 학습되는지  
(한국어로 자세히 설명합니다)

---

### 1. 전체 흐름 개요  

| 단계 | 목적 | 주요 기법 |
|------|------|----------|
| **데이터 수집·정제** | 다양한 언어·주제의 텍스트 확보 | 웹 크롤링, 책·논문·위키피디아·코드 레포 등 |
| **토크나이징** | 텍스트를 모델이 이해할 수 있는 **토큰**(기호)으로 변환 | BPE(Byte‑Pair Encoding)·WordPiece 등 |
| **사전 학습 (Pre‑training)** | **다음 토큰 예측**이라는 목표로 언어 구조와 지식 습득 | 트랜스포머(Transformer) 아키텍처, 언어 모델링 손실(Cross‑Entropy) |
| **지도 학습 (Supervised Fine‑tuning)** | 사용자가 기대하는 형태(질문‑답변, 지시 수행 등)로 모델을 조정 | 인간이 만든 프롬프트‑응답 쌍을 이용 |
| **보상 모델 학습 (Reward Model)** | 인간 피드백을 수치화해 “좋은 답변”을 정의 | 인간 라벨러가 평가한 답변을 기반으로 랭킹 모델 학습 |
| **RLHF (Reinforcement Learning from Human Feedback)** | 보상 모델을 이용해 정책을 최적화 → **안전하고 유용한** 응답 생성 | P

In [7]:
# 체인을 생성하여 호출하기
#llm = ChatOpenAI(model="gpt-3.5-turbo-0125")

chain = chat_prompt | llm | StrOutputParser()

response = chain.invoke({"topic":"AI", "model_name":"ChatGPT"})
print(type(response))
print(response)

<class 'str'>
## ChatGPT 모델의 학습 원리

ChatGPT는 **대규모 언어 모델**(Large Language Model, LLM)이며, 기본 구조는 **Transformer**라는 딥러닝 아키텍처에 기반합니다. 아래에서는 데이터 준비부터 사전 학습(pre‑training), 미세 조정(fine‑tuning), 그리고 실제 서비스에 적용되는 **RLHF**(Reinforcement Learning from Human Feedback)까지 전체 학습 파이프라인을 단계별로 살펴보겠습니다.

---

### 1. 모델 구조: Transformer 디코더

| 구성 요소 | 역할 |
|----------|------|
| **입력 임베딩** (Token Embedding) | 텍스트를 정수 토큰으로 변환하고, 각 토큰을 고정 차원의 벡터로 매핑 |
| **포지셔널 인코딩** (Positional Encoding) | 순서 정보를 제공 (Transformer 자체는 순서를 인식하지 않으므로) |
| **멀티‑헤드 셀프‑어텐션** (Multi‑Head Self‑Attention) | 각 토큰이 문맥 안의 모든 다른 토큰과 어떻게 연관되는지를 학습 |
| **피드‑포워드 네트워크** (Feed‑Forward) | 비선형 변환을 통해 표현력을 확대 |
| **레이어 정규화 & 잔차 연결** (LayerNorm + Residual) | 학습 안정성과 깊은 네트워크 훈련을 돕는다 |
| **출력 소프트맥스** (Softmax) | 다음에 올 토큰의 확률 분포를 계산 |

이 구조를 **N개의 레이어**(예: GPT‑3에서는 96층, GPT‑4에서는 더 많음)로 쌓아 깊고 풍부한 언어 표현을 얻습니다.

---

### 2. 사전 학습 (Pre‑training) – **자기지도 학습 (Self‑Supervised Learning)**

1. **데이터 수집**  
   - 인터넷에 공개된 웹 페이지, 책, 논문, 위키피디아, 포럼 글 등 방대한 텍스트 코퍼스

#### 3) ChatPromptTemplate
* SystemMessagePromptTemplate와 HumanMessagePromptTemplate 클래스 사용
* 객체 지향적 접근 - Message 객체를 독립적으로 생성 가능
* 여러 조건에 따라 다른 시스템 메시지 선택

```python
if user_is_beginner:
    system_message = SystemMessagePromptTemplate.from_template("초보자를 위한 설명: {topic}")
else:
    system_message = SystemMessagePromptTemplate.from_template("전문가를 위한 상세 분석: {topic}")
```

In [ ]:
# ChatMessagePromptTemplate 활용

from langchain_core.prompts import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    AIMessagePromptTemplate,
    ChatMessagePromptTemplate
)
from langchain_openai import ChatOpenAI

# 개별 메시지 템플릿 정의
system_message = SystemMessagePromptTemplate.from_template(
    "You are an AI expert in {topic}. Please provide clear and detailed explanations."
)
user_message = HumanMessagePromptTemplate.from_template(
    "{question}"
)
ai_message = AIMessagePromptTemplate.from_template(
    "This is an example answer about {topic}."
)

# ChatPromptTemplate로 메시지들을 묶기
chat_prompt = ChatPromptTemplate.from_messages([
    system_message,
    user_message,
    ai_message
])

# 메시지 생성
messages = chat_prompt.format_messages(topic="AI", question="What is deep learning?")

# LLM 호출
#llm = ChatOpenAI(model="gpt-3.5-turbo-0125")
response = llm.invoke(messages)

# 결과 출력
print(response.content)

#### ChatMessagePromptTemplate는 여러 종류의 메시지(시스템, 인간, AI)를 조합하여 복잡한 프롬프트를 생성할 때 유용합니다.
* SystemMessagePromptTemplate: 이 템플릿은 AI 모델에게 역할을 부여하거나 전반적인 규칙을 설정하는 시스템 메시지를 만듭니다. 위의 예시에서는 "번역을 도와주는 유용한 도우미"라는 역할을 지정합니다.
* HumanMessagePromptTemplate: 이 템플릿은 사용자의 질문이나 요청을 담는 인간 메시지를 만듭니다. 아래의 예시에서는 번역할 텍스트를 입력받습니다.
* ChatPromptTemplate.from_messages: 이 클래스 메서드는 시스템 메시지, 인간 메시지 등 여러 종류의 MessagePromptTemplate 객체들을 리스트로 받아 하나의 채팅 프롬프트 템플릿으로 통합합니다.
* format_messages: 이 메서드는 정의된 템플릿에 실제 값을 채워 넣어 [SystemMessage, HumanMessage] 형태의 리스트를 반환합니다. 이 리스트는 채팅 모델(Chat Model) 에 바로 전달될 수 있습니다.

In [ ]:
# 필요한 라이브러리 임포트
from langchain.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate

# 1. SystemMessagePromptTemplate와 HumanMessagePromptTemplate 생성
# SystemMessagePromptTemplate는 모델의 페르소나 또는 기본 지침을 설정합니다.
system_template = "You are a helpful assistant that translates {input_language} to {output_language}."
system_message_prompt = SystemMessagePromptTemplate.from_template(system_template)

# HumanMessagePromptTemplate는 사용자로부터 받는 입력 프롬프트를 정의합니다.
human_template = "{text_to_translate}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

# 2. ChatPromptTemplate 생성
# 위에서 만든 두 템플릿을 리스트로 묶어 ChatPromptTemplate을 만듭니다.
chat_prompt_template = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])

# 3. 프롬프트 포맷팅
# chat_prompt_template.format_messages()를 사용하여 최종 메시지 리스트를 생성합니다.
# 이 함수는 딕셔너리 형태의 입력 변수를 받습니다.
formatted_prompt = chat_prompt_template.format_messages(
    input_language="English",
    output_language="Korean",
    text_to_translate="I love programming."
)

# 4. 결과 출력
print(formatted_prompt)

# LLM 호출
response = llm.invoke(formatted_prompt)

# 결과 출력
print(response.content)

##### 4) FewShotPromptTemplate
* FewShotPromptTemplate은 모델이 특정 형식을 따르게 하거나, 일관된 응답을 생성하도록 유도할 때 유용합니다.
* 도메인 지식이 필요하거나, AI가 오답을 줄이고 더 신뢰할 만한 답변을 생성하도록 해야 할 때 효과적입니다.

##### 4-1) PromptTemplate을 사용하지 않는 경우

In [19]:
# PromptTemplate을 사용하지 않는 경우
from langchain_openai import ChatOpenAI

# model
#llm = ChatOpenAI(model="gpt-3.5-turbo")

# chain 실행
result = llm.invoke("태양계의 행성들을 간략히 정리해 주세요.")

print(type(result))
print(result.content)

<class 'langchain_core.messages.ai.AIMessage'>
태양계에는 8개의 행성이 있습니다. 

1.  수성: 태양과 가장 가까운 행성으로, 표면이 암석으로 구성되어 있고 극도로 높은 온도와 낮은 온도가 반복됩니다.
2.  금성: 태양계에서 두 번째로 가까운 행성으로, 두꺼운 대기로 인해 극심한 온실 효과가 발생하여 매우 뜨겁습니다.
3.  지구: 우리가 사는 행성으로, 물과 대기가 있어 생명체가 존재할 수 있습니다.
4.  화성: 태양계에서 네 번째로 가까운 행성으로, 붉은색의 모래사막으로 덮여 있고 물과 생명체의 존재 가능성이 있습니다.
5.  목성: 태양계에서 가장 큰 행성으로, 가스 거인이며 강력한 자기장과 수많은 위성을 가지고 있습니다.
6.  토성: 태양계에서 두 번째로 큰 행성으로, 가스 거인이며 아름다운 고리를 가지고 있습니다.
7.  천왕성: 태양계에서 일곱 번째로 가까운 행성으로, 가스 거인이며 자전축이 기울어져 있어 극단적인 기후 변화를 경험합니다.
8.  해왕성: 태양계에서 가장 먼 행성으로, 가스 거인이며 강한 바람과 극적인 기후 변화를 가지고 있습니다.

이러한 행성들은 각각 고유한 특징과 성질을 가지고 있으며, 태양계에서 중요한 역할을 합니다.


##### 4-2) FewShotChatMessagePromptTemplate 사용하는 경우

In [9]:
# FewShotChatMessagePromptTemplate 사용하는 경우
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate
from langchain_openai import ChatOpenAI

examples = [
    {
        "input": "뉴턴의 운동 법칙을 요약해 주세요.",
        "output": """### 뉴턴의 운동 법칙
1. **관성의 법칙**: 힘이 작용하지 않으면 물체는 계속 같은 상태를 유지합니다.
2. **가속도의 법칙**: 물체에 힘이 작용하면, 힘과 질량에 따라 가속도가 결정됩니다.
3. **작용-반작용 법칙**: 모든 힘에는 크기가 같고 방향이 반대인 힘이 작용합니다."""
    },
    {
        "input": "지구의 대기 구성 요소를 알려주세요.",
        "output": """### 지구 대기의 구성
- **질소 (78%)**: 대기의 대부분을 차지합니다.
- **산소 (21%)**: 생명체가 호흡하는 데 필요합니다.
- **아르곤 (0.93%)**: 반응성이 낮은 기체입니다.
- **이산화탄소 (0.04%)**: 광합성 및 온실 효과에 중요한 역할을 합니다."""
    }
]

# 예제 프롬프트 템플릿
example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}"),
        ("ai", "{output}"),
    ]
)

# FewShotChatMessagePromptTemplate 적용
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

# 최종 프롬프트 구성
final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "당신은 초등학생도 쉽게 이해할 수 있도록 쉽게 설명하는 과학 교육자입니다."),
        few_shot_prompt,
        ("human", "{input}"),
    ]
)

# 모델 생성 및 체인 구성
#llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.0)
chain = final_prompt | llm
print(chain)

# 테스트 실행
result = chain.invoke({"input": "태양계의 행성들을 간략히 정리해 주세요."})
print(result.content)

first=ChatPromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='당신은 초등학생도 쉽게 이해할 수 있도록 쉽게 설명하는 과학 교육자입니다.'), additional_kwargs={}), FewShotChatMessagePromptTemplate(examples=[{'input': '뉴턴의 운동 법칙을 요약해 주세요.', 'output': '### 뉴턴의 운동 법칙\n1. **관성의 법칙**: 힘이 작용하지 않으면 물체는 계속 같은 상태를 유지합니다.\n2. **가속도의 법칙**: 물체에 힘이 작용하면, 힘과 질량에 따라 가속도가 결정됩니다.\n3. **작용-반작용 법칙**: 모든 힘에는 크기가 같고 방향이 반대인 힘이 작용합니다.'}, {'input': '지구의 대기 구성 요소를 알려주세요.', 'output': '### 지구 대기의 구성\n- **질소 (78%)**: 대기의 대부분을 차지합니다.\n- **산소 (21%)**: 생명체가 호흡하는 데 필요합니다.\n- **아르곤 (0.93%)**: 반응성이 낮은 기체입니다.\n- **이산화탄소 (0.04%)**: 광합성 및 온실 효과에 중요한 역할을 합니다.'}], input_variables=[], input_types={}, partial_variables={}, example_prompt=ChatPromptTemplate(input_variables=['input', 'output'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=[

#### 5-1) PartialPrompt 
* 프롬프트를 더 동적으로 활용할 수 있으며, AI 응답을 더 일관성 있게 조정 가능함

In [ ]:
from datetime import datetime
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

# 계절을 결정하는 함수 (남반구/북반구 고려)
def get_current_season(hemisphere="north"):
    month = datetime.now().month

    if hemisphere == "north":  # 북반구 (기본값)
        if 3 <= month <= 5:
            return "봄"
        elif 6 <= month <= 8:
            return "여름"
        elif 9 <= month <= 11:
            return "가을"
        else:
            return "겨울"
    else:  # 남반구 (계절 반대)
        if 3 <= month <= 5:
            return "가을"
        elif 6 <= month <= 8:
            return "겨울"
        elif 9 <= month <= 11:
            return "봄"
        else:
            return "여름"

# 프롬프트 템플릿 정의 (부분 변수 적용)
prompt = PromptTemplate(
    template="{season}에 일어나는 대표적인 지구과학 현상은 {phenomenon}이 맞나요? {season}에 주로 발생하는 지구과학 현상을 3개 알려주세요",
    input_variables=["phenomenon"],  # 사용자 입력 필요
    partial_variables={"season": get_current_season()}  # 동적으로 계절 값 할당
)

# OpenAI 모델 초기화
#llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.0)

# 특정 계절의 현상 질의
query = prompt.format(phenomenon="태풍 발생")
result = llm.invoke(query)


# 결과 출력
print(f" 프롬프트: {query}")
print(f" 모델 응답: {result.content}")

In [ ]:
from datetime import datetime
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

# 계절을 결정하는 함수 (남반구/북반구 고려)
def get_current_season(hemisphere="north"):
    month = datetime.now().month

    if hemisphere == "north":  # 북반구 (기본값)
        if 3 <= month <= 5:
            return "봄"
        elif 6 <= month <= 8:
            return "여름"
        elif 9 <= month <= 11:
            return "가을"
        else:
            return "겨울"
    else:  # 남반구 (계절 반대)
        if 3 <= month <= 5:
            return "가을"
        elif 6 <= month <= 8:
            return "겨울"
        elif 9 <= month <= 11:
            return "봄"
        else:
            return "여름"

# Step 1: 현재 계절 결정
season = get_current_season("north")  # 계절 값 얻기
print(f"현재 계절: {season}")

# Step 2: 해당 계절의 자연 현상 추천
prompt2 = ChatPromptTemplate.from_template(
    "{season}에 주로 발생하는 대표적인 지구과학 현상 3가지를 알려주세요. "
    "각 현상에 대해 간단한 설명을 포함해주세요."
)

# OpenAI 모델 사용
#llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.0)
llm = ChatOpenAI(
    #api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",  # Spring AI와 동일한 모델
    temperature=0.0
)

# 체인 2: 자연 현상 추천 (입력: 계절 → 출력: 자연 현상 목록)
chain2 = (
    {"season": lambda x : season}  # chain1의 출력을 season 변수로 전달
    | prompt2
    | llm
    | StrOutputParser()
)

# 실행: 현재 계절에 따른 자연 현상 추천
response = chain2.invoke({})
print(f"\n {season}에 발생하는 자연 현상:\n{response}")

#### 5-2) PartialPrompt 
* API 호출 데이터, 시간 정보, 사용자 정보 등을 반영할 때 매우 유용함

In [10]:
import requests
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

# 실시간 환율을 가져오는 함수
def get_exchange_rate():
    response = requests.get("https://api.exchangerate-api.com/v4/latest/USD")
    data = response.json()
    return f"1달러 = {data['rates']['KRW']}원"

# Partial Prompt 활용
prompt = PromptTemplate(
    template="현재 {info} 기준으로 환율 정보를 알려드립니다. 이에 대한 분석을 제공해 주세요.",
    input_variables=[],  # 사용자 입력 없음
    partial_variables={"info": get_exchange_rate()}  # API에서 가져온 데이터 자동 반영
)

# LLM 모델 설정
#llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.0)

# 모델에 프롬프트 전달 및 응답 받기
response = llm.invoke(prompt.format())

# 결과 출력
print(" 프롬프트:", prompt.format())
print(" 모델 응답:", response.content)

 프롬프트: 현재 1달러 = 1377.98원 기준으로 환율 정보를 알려드립니다. 이에 대한 분석을 제공해 주세요.
 모델 응답: ### 1 USD = 1,377.98 KRW 환율에 대한 종합 분석

| 구분 | 내용 |
|------|------|
| **현재 환율** | 1 USD = 1,377.98 KRW (2025‑09‑18 기준) |
| **주요 변동 요인** | • 미국 연방준비제도(Fed)·한국은행(BOK) 금리 차이<br>• 양국 물가·인플레이션 전망<br>• 글로벌 위험 선호도·달러 강세/약세 흐름<br>• 한국 무역수지·수출입 구조 변화 |
| **최근 12개월 추이** | - 2024 초: 1,300 KRW 수준 (달러 약세)<br>- 2024 중반: 1,350 KRW까지 상승 (미국 금리 인상 지속)<br>- 2024 말~2025 초: 1,380 KRW 수준 유지 (한국 금리 인하 기대감 감소)<br>- 현재 1,378 KRW는 연간 평균 대비 약 +2 % 수준 |
| **역사적 비교** | - 2000년대 초반: 1,200 KRW 수준 (달러 약세)<br>- 2008년 금융위기 이후: 1,100 KRW 수준 (달러 강세)<br>- 2010‑2015: 1,100~1,200 KRW (안정기)<br>- 2020‑2022: 1,150~1,300 KRW (코로나·미국 금리 인상)<br>현재는 **역사적 고점(2023 말 1,400 KRW) 근처**에 해당합니다. |

---

## 2. 환율 변동에 영향을 미치는 주요 거시경제 요인

| 요인 | 현재 상황 | 환율에 미치는 영향 |
|------|----------|-------------------|
| **미국 금리 정책** | 연방준비제도(Fed) 금리 **5.25 %** 수준 유지, 인플레이션 억제 목표로 추가 인상 가능성 낮음 | 높은 금리는 달러 매력을 유지 → 원·달러 환율 상승 압력 |
| **한국 금리 정책** | 한국은행 기준금리 **3.50 %** (2025 Q2) – 물가 상